In [5]:
# import libraries 
import tensorflow as tf

# Image Agumentation with Preprocessing

In [6]:
train_images = r"/home/ariel/Documents/Face Images/Final Training Images"
train_images

'/home/ariel/Documents/Face Images/Final Training Images'

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
train_gen = ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

In [10]:
# No transformations are made on the test data
test_gen = ImageDataGenerator()

# Generating training data
training_data = train_gen.flow_from_directory(
    train_images, 
    target_size = (100,100),
    batch_size = 30,
    class_mode = 'categorical'
)

Found 244 images belonging to 16 classes.


In [11]:
# generating test data
testing_data = test_gen.flow_from_directory(
    train_images, 
    target_size = (100,100),
    batch_size = 30,
    class_mode = 'categorical'
)

# Printing class labels for each face
testing_data.class_indices

Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

In [12]:
# training_data have numeric tag for each face
Train_class = training_data.class_indices

# lets store them in a dictionary with swap for future reference
Result_class = {}
for value_tag, face_tag in zip(Train_class.values(),Train_class.keys()):
    Result_class[value_tag] = face_tag

    
# use pickle to save the mapping's
import pickle5 as pickle
with open(r'/home/ariel/Documents/Face Images/ResultMap.pkl','wb') as Final_mapping:
    pickle.dump(Result_class,Final_mapping)

In [13]:
print("Mapping of Face and its numeric value",Result_class)

Mapping of Face and its numeric value {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}


In [14]:
Output_Neurons=len(Result_class)
print('\n The Number of output neurons: ', Output_Neurons)


 The Number of output neurons:  16


# Building the CNN Architecture, Model Compilation and Training

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [16]:
'''Initializing the Convolutional Neural Network'''
Model = Sequential()

In [17]:
''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (100,100,3) because we are using TensorFlow backend
# It means 3 matrix of size (100x100) pixels representing Red, Green and Blue components of pixels
'''

Model.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), input_shape = (100,100,3),activation='relu'))

/home/ariel/PycharmProjects/enronmails/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1731067174.862912   12218 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [18]:
'''# Maxplooing layer'''

Model.add(MaxPool2D(pool_size=(2,2)))

In [19]:
'''Adding additional layers  of convolution and 
        maxpooling for better model accuracy and performance'''

Model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu'))
Model.add(MaxPool2D(pool_size=(2,2)))

'''# Add a flatten layer to convert the vector to one dimensional'''
Model.add(Flatten())

In [20]:
'''Add dense layers and Initialize weights using 
                kernal initializer for better learing of image features and classification'''

Model.add(Dense(64,activation='relu'))
Model.add(Dense(Output_Neurons,activation='softmax'))

In [28]:
Model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 96, 96, 16)     │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 48, 48, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 46, 46, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16928)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,083,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,271,058 (12.48 MB)

 Trainable params: 1,090,352 (4.16 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,180,706 (8.32 MB)

In [30]:
'''Perform Model Compilation'''

Model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics = ['Accuracy'])

In [22]:
'''# Using Early stopping to reduce the training time'''

from keras.callbacks import EarlyStopping
call = EarlyStopping(
                    min_delta=0.005,
                    patience=5,
                     verbose=1
                    )

In [32]:
# Model Training
Model.fit(
    training_data,
    batch_size = 16,
    epochs=30,
    validation_data=testing_data
)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 215ms/step - Accuracy: 0.1022 - loss: 2.7648 - val_Accuracy: 0.0820 - val_loss: 2.7678
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - Accuracy: 0.0854 - loss: 2.7676 - val_Accuracy: 0.0820 - val_loss: 2.7678
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 192ms/step - Accuracy: 0.0970 - loss: 2.7642 - val_Accuracy: 0.0820 - val_loss: 2.7677
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - Accuracy: 0.0953 - loss: 2.7687 - val_Accuracy: 0.0820 - val_loss: 2.7676
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - Accuracy: 0.0761 - loss: 2.7686 - val_Accuracy: 0.0820 - val_loss: 2.7676
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 189ms/step - Accuracy: 0.0688 - loss: 2.7714 - val_Accuracy: 0.0820 - val_loss: 2.7675
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 192ms/step - Accuracy: 0.0834 - loss: 2.7694 - val_Accuracy: 0.0820 - val_loss: 2.7675
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - Accuracy: 0.0777 - loss: 2.7682 - val_Accuracy: 0.0820 - val_loss: